# Simulating a 3D Lennard-Jones Fluid at NVT

## Model

Distribute particles along a 3D lattice and initialize velocities according to a Boltzmann Distribution at starting temp Ti. Shift velocities so net velocitiy (center of mass velocity) is 0

Use velocity Verlet algorithm for MD steps

Calculate force by summing over all pairs of particles

Assume periodic boundry conditions and cutoff distance($ r_{c} $) < 1/2 diameter of box($ d_{box} $) so interactions are only between nearest periodic image. Therefore: $$ d_{x}(i,j) = x(i) - x(j) - d_{box} \cdot round\left(\frac{x(i) - x(j)}{d_{box}}\right) $$

Lennard-Jones Potential Energy: $$ V(r) = 4 \epsilon \left(\left(\frac{\sigma}{r}\right)^{12}-\left(\frac{\sigma}{r}\right)^{6}\right) = 4 \left(\frac{1}{r^{*12}} - \frac{1}{r^{*6}}\right) = \frac{4}{r^{*6}} \left(\frac{1}{r^{*6}} - 1\right) $$

and Lennard-Jones Force: $$ F_{x}(r) = -\frac{\partial}{\partial x} \cdot V(r) = -\left(\frac{d_{x}}{r}\right)\left(\frac{\partial}{\partial r} \cdot V(r)\right) = \frac{48 d_{x}}{r^{*2}}\left(\frac{1}{r^{*12}} - 0.5\frac{1}{r^{*6}}\right) = \frac{48 d_{x}}{r^{*2}} \cdot \frac{1}{r^{*6}}\left(\frac{1}{r^{*6}} - 0.5\right) $$

where x = any cartesian axis, $ d_{x} $ = distance between particles i and j along axis x, and r = vector distance between particles i and j. Therefore in 3D space:

$$ r = \sqrt{d_{x}^{2} + d_{y}^{2} + d_{z}^{2}} $$

$$ V(d_{x}, d_{y}, d_{z}) = \frac{4}{\left(d_{x}^{2} + d_{y}^{2} + d_{z}^{2}\right)^{3}} \left(\frac{1}{\left(d_{x}^{2} + d_{y}^{2} + d_{z}^{2}\right)^{3}} - 1\right) $$

$$ F_{x}(d_{x}, d_{y}, d_{z}) = \frac{48 d_{x}}{d_{x}^{2} + d_{y}^{2} + d_{z}^{2}} \cdot \frac{1}{\left(d_{x}^{2} + d_{y}^{2} + d_{z}^{2}\right)^{3}}\left(\frac{1}{\left(d_{x}^{2} + d_{y}^{2} + d_{z}^{2}\right)^{3}} - 0.5\right) $$

Position: $$ x(t + dt) = x(t) + v(t)dt + \frac{F(t)}{2m}dt^{2} $$

Velocity: $$ v(t + dt) = v(t) + \frac{f(t + dt) + f(t)}{2m}dt = v(t) + \frac{f(t + dt)}{2m}dt + \frac{f(t)}{2m}dt = v(t + \frac{1}{2}dt) + \frac{f(t + dt)}{2m}dt $$

where $$ v(t + \frac{1}{2}dt) = v(t) + \frac{f(t)}{2m}dt $$

## Initialization

Imports:

In [1]:
%matplotlib inline
#%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'
import matplotlib.pyplot as plt
import numpy as np

Global Constants:

In [59]:
rng = np.random.default_rng()
m = 1
dt = 0.01
cycles = 1000
ndim = 5
nparts = ndim**3
Ti = 2
dbox = 5

## Definitions

In [60]:
#returns position after time delt using the velocity verlet method
def verPos(currPos, vel, frc):
    return currPos + (vel * dt) + (frc * dt**2 / 2 / m)

#returns velocity after time delt using the velocity verlet method with half timestep
def verVel(currVel, frc):
    return currVel + (dt * frc / 2 / m)

#returns distance between particle i and j along x axis 
def dist(xi, xj):
    return xi - xj - dbox * round((xi - xj) / dbox)

#returns 3D vector distance from distance components dx, dy, and dz
def vdist(dx, dy, dz):
    return np.sqrt(dx**2 + dy**2 + dz**2)

#returns LJ force along x axis
def force(dx, r):
    return 48 * dx * r**-2 * r**-6 * (r**-6 - 0.5)

#reutrns LJ potential
def potE(r):
    return 4 * r**-6 * (r**-6 - 1)

#Calculates forces across all pairs of particles
def forces(frc, pos):
    frc = np.zeros_like(frc)
    for i in range(nparts - 1):
        for j in range(i + 1, nparts):
            dx = dist(pos[i,0], pos[j,0])
            dy = dist(pos[i,1], pos[j,1])
            dz = dist(pos[i,2], pos[j,2])
            r = vdist(dx, dy, dz)
            frc[i,0] += force(dx, r)
            frc[i,1] += force(dy, r)
            frc[i,2] += force(dz, r)
            frc[j,0] -= force(dx, r)
            frc[j,1] -= force(dy, r)
            frc[j,2] -= force(dz, r)
    return frc

#Calculates new positions
def positions(pos, vel, frc):
    for i in range(nparts):
        for j in range(3):
            pos[i,j] = verPos(pos[i,j], vel[i,j], frc[i,j])
    return pos

#Calculates new velocities
def velocities(vel, frc):
    for i in range(nparts):
        for j in range(3):
            vel[i,j] = verVel(vel[i,j], frc[i,j])
    return vel

## Simulation

In [61]:
#Creating the arrays
Pos = np.zeros([nparts, 3])
Vel = np.zeros([nparts, 3])
Frc = np.zeros([nparts, 3])

#Filling the arrays with the initial conditions
Pos = np.vstack(np.meshgrid(np.linspace(-dbox/2,dbox/2,ndim),np.linspace(-dbox/2,dbox/2,ndim),np.linspace(-dbox/2,dbox/2,ndim))).reshape(3,-1).T
for i in range(3):
    sumV = 0
    for j in range(nparts):
        Vel[j,i] = rng.normal(0, np.sqrt(Ti))
        sumV += Vel[j,i]
    sumV = sumV / nparts
    for j in range(nparts):
        Vel[j,i] = Vel[j,i] - sumV

#MD Loop
for i in range(cycles):
    Frc = forces(Frc, Pos)
    Pos = positions(Pos, Vel, Frc)
    Vel = velocities(Vel, Frc)
    Frc = forces(Frc, Pos)
    Vel = velocities(Vel, Frc)
